In [0]:
!pip install -q tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 35kB/s 
     |████████████████████████████████| 3.9MB 40.4MB/s 
     |████████████████████████████████| 450kB 40.3MB/s 


# **Tensorflow 1.0 in Tensorflow 2.0**

In [0]:
import tensorflow.compat.v1 as tf


In [0]:
in_a = tf.placeholder(dtype=tf.float32, shape=(2))

In [0]:
tf.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [0]:
def model(x):
  with tf.variable_scope("matmul"):
    W = tf.get_variable("W", initializer=tf.ones(shape=(2,2)))
    b = tf.get_variable("b", initializer=tf.zeros(shape=(2)))
    return x * W + b

In [0]:
out_a = model(in_a)

In [0]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  outs = sess.run([out_a], feed_dict={in_a: [1, 0]})
  writer = tf.summary.FileWriter("./logs/example", sess.graph)

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [0]:
%tensorboard --logdir 

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2020-04-07 04:35:34.339857: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-04-07 04:35:34.340005: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-04-07 04:35:34.340020: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, 

#**AutoGraph**

In [0]:
import tensorflow as tf

In [0]:
def linear_layer(x):
  return 3 * x + 2

@tf.function
def simple_nn(x):
  return tf.nn.relu(linear_layer(x))

def simple_function(x):
  return 3 * x

# internal look at the autogenerated code
print(tf.autograph.to_code(simple_nn.python_function, experimental_optional_features=None))

def tf__simple_nn(x):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('simple_nn', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
    do_return = True
    retval_ = fscope.mark_return_value(ag__.converted_call(tf.nn.relu, (ag__.converted_call(linear_layer, (x,), None, fscope),), None, fscope))
  do_return,
  return ag__.retval(retval_)



In [0]:
import timeit

In [0]:
cell = tf.keras.layers.LSTMCell(100)

@tf.function
def fn(input, state):
  return cell(input, state)

input = tf.zeros([100, 100])
state = [tf.zeros([100, 100])] * 2

#warmup
cell(input, state)
fn(input, state)

graph_time = timeit.timeit(lambda: cell(input, state), number=100)
auto_graph_time = timeit.timeit(lambda: fn(input, state), number=100)
print('graph_time:', graph_time)
print('auto_graph_time:', auto_graph_time)

graph_time: 0.1342246829999567
auto_graph_time: 0.10669438400054787


In [0]:
def build_model():
  # Variable-length sequence of integers
  text_input_a = tf.keras.Input(shape=(None,), dtype='int32')

  # Variable-length sequence of integers
  text_input_b = tf.keras.Input(shape=(None,), dtype='int32')

  #Embedding for 100 unique words mapped to 128-dimensional vectors
  shared_embedding = tf.keras.layers.Embedding(1000, 128)

  # We reuse the same layer to encode both inputs
  encoded_input_a = shared_embedding(text_input_a)
  encoded_input_b = shared_embedding(text_input_b)

  # two logistic predictions at the end
  prediction_a = tf.keras.layers.Dense(1, activation='sigmoid', name='prediction_a')(encoded_input_a)
  prediction_b = tf.keras.layers.Dense(1, activation='sigmoid', name='prediction_b')(encoded_input_b)

  # this model 2 inputs, and 2 outputs
  # in the middle, we have a shared model
  model = tf.keras.Model(inputs=[text_input_a, text_input_b], outputs=[prediction_a, prediction_b])
  tf.keras.utils.plot_model(model, to_file="shared_model.png")


In [0]:
build_model()

#**Model Subclassing**

In [0]:
class MyLayer(tf.keras.layers.Layer):
  def __init__(self, output_dim, **kwargs):
    self.output_dim = output_dim
    super(MyLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    # Create a trainable weight variable for this layer
    self.kernel = self.add_weight(name='kernel', shape=(input_shape[1], self.output_dim), initializer='uniform', trainable=True)

  def call(self, inputs):
    # Do the multiplication and return
    return tf.matmul(inputs, self.kernel)


In [0]:
model = tf.keras.Sequential([MyLayer(20), tf.keras.layers.Activation('softmax')])

# **Datasets**

In [0]:
import tensorflow_datasets as tfds

In [0]:
# See all registered datasets
builders = tfds.list_builders()
print(builders)

['abstract_reasoning', 'aeslc', 'aflw2k3d', 'amazon_us_reviews', 'arc', 'bair_robot_pushing_small', 'beans', 'big_patent', 'bigearthnet', 'billsum', 'binarized_mnist', 'binary_alpha_digits', 'c4', 'caltech101', 'caltech_birds2010', 'caltech_birds2011', 'cars196', 'cassava', 'cats_vs_dogs', 'celeb_a', 'celeb_a_hq', 'cfq', 'chexpert', 'cifar10', 'cifar100', 'cifar10_1', 'cifar10_corrupted', 'citrus_leaves', 'cityscapes', 'civil_comments', 'clevr', 'cmaterdb', 'cnn_dailymail', 'coco', 'coil100', 'colorectal_histology', 'colorectal_histology_large', 'cos_e', 'curated_breast_imaging_ddsm', 'cycle_gan', 'deep_weeds', 'definite_pronoun_resolution', 'diabetic_retinopathy_detection', 'div2k', 'dmlab', 'downsampled_imagenet', 'dsprites', 'dtd', 'duke_ultrasound', 'dummy_dataset_shared_generator', 'dummy_mnist', 'emnist', 'eraser_multi_rc', 'esnli', 'eurosat', 'fashion_mnist', 'flic', 'flores', 'food101', 'gap', 'gigaword', 'glue', 'groove', 'higgs', 'horses_or_humans', 'i_naturalist2017', 'image

In [0]:
# Load a given dataset by name, along with the DatasetInfo metadata
data, info = tfds.load("mnist", with_info=True)
train_data, test_data = data['train'], data['test']

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


In [0]:
print(info)

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.0,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)



In [0]:
import numpy as np

In [0]:
num_items = 100
num_list = np.arange(num_items)
print(num_list)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]


In [0]:
# create the dataset from numpy array
num_list_dataset = tf.data.Dataset.from_tensor_slices(num_list)
print(num_list_dataset)

<TensorSliceDataset shapes: (), types: tf.int64>


In [0]:
type(num_list_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [0]:
num_list_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

In [0]:
# Download dataset, then batch and shuffle the data. 
datasets, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_dataset = datasets['train']
train_dataset = train_dataset.batch(5).shuffle(50).take(2)

for data in train_dataset:
  print(data)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJM4WO8/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJM4WO8/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJM4WO8/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
(<tf.Tensor: shape=(5,), dtype=string, numpy=
array([b"This movie was bad to say the least!!! The plethora of superb cars are disgraced to have have been showcased in this LAME movie. It starts off with a race from L.A. to Las Vegas to be completed in 1HR 45min...in a Ferrari F430. I did that in 1HR 50min in a tiny 4cylinder 140HP 1993 Honda Accord. Seriously...this movie does not do justice to these cars. Obviously these writers are just that and probably drive under the speed limit with their hands @ 10 & 2 o'clock. I remember seeing on the news how Eddie Griffin crashed a 1.5 million dollar Ferrari Enzo going 30-40MPH>>> youtube.com/watch?v=cNVrMZX2kms <br /><br />And...the director ANDY CHENG is THE biggest SELLOUT!!! He brings shame to his own race. I wonder just how many people he orally pleased just to break into Hollywood. He partook in

# **Batch Datasets**

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [0]:
builder = tfds.builder('imdb_reviews')
builder.download_and_prepare()

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWOLUWW/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWOLUWW/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWOLUWW/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
datasets, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

train_dataset = datasets['train']
train_dataset = train_dataset.batch(5).shuffle(50).take(2)

In [0]:
for data in train_dataset:
  print(data)

(<tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'You remember the Spice Girls movie and how bad it was (besides the songs), well their manager Simon Fuller (also this band\'s manager) makes the same error putting S Club (another of my favourite bands) in their own film. S Club: Tina Barrett, Jon Lee, Bradley Mcintosh, Jo O\'Meara, Hannah Spearritt and Rachel Stevens (what happened to the seventh member, Paul Cattermole?) basically ask their boss for a break, they go on, and while there they see themselves on TV! Three of them swap, and vice versa, half discover they are clones made by a greedy scientist, and the other half just get themselves in trouble. Also starring Gareth Gates as a clone of himself. This film may have more of a plot than Spice Girls\' film did, but besides the songs "Bring It All Back", "Don\'t Stop Movin\'" and "Never Had A Dream Come True" this is no goo reason to see this film. Not too long after the band split for good. Adequate!',
       b"Recap: Full mo

# **Estimators**

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [0]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [0]:
def input_fn(mode):
  datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
  mnist_dataset = (datasets['train']  if mode == tf.estimator.ModeKeys.TRAIN else datasets['test'])

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

  return mnist_dataset.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [0]:
test = input_fn('test')
train = input_fn(tf.estimator.ModeKeys.TRAIN)

print(test)
print(train)

<DatasetV1Adapter shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>
<DatasetV1Adapter shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>


# **Gradient Tape**

In [0]:
import tensorflow as tf

x = tf.constant(4.0)

with tf.GradientTape(persistent=True) as g:
  g.watch(x)
  y = x * x
  z = y * y
  dz_dx = g.gradient(z, x) # 108.0 
  dy_dx = g.gradient(y, x) # 6.0

  print(dz_dx)
  print(dy_dx)
  del g # Drop the reference of the tape

tf.Tensor(256.0, shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)
